In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
import os
import copy
import matplotlib.pyplot as plt

from __future__ import print_function, division
import time

plt.ion()

In [ ]:
from typing import Union, Callable, Tuple
from functools import reduce
from collections import deque
from typing import Union, Tuple
from torch.nn import Module

def _iterative_gradient(model: Module,
                        x: torch.Tensor,
                        y: torch.Tensor,
                        loss_fn: Callable,
                        k: int,
                        step: float,
                        eps: float,
                        norm: Union[str, float],
                        step_norm: Union[str, float],
                        y_target: torch.Tensor = None,
                        random: bool = False) -> torch.Tensor:
  
    x_adv = x.clone().detach().requires_grad_(True).to(x.device)
    targeted = y_target is not None

    if random:
        x_adv = random_perturbation(x_adv, norm, eps)

    for i in range(k):
        _x_adv = x_adv.clone().detach().requires_grad_(True)

        prediction = model(_x_adv)
        loss = loss_fn(prediction, y_target if targeted else y)
        loss.backward()

        with torch.no_grad():
            if step_norm == 'inf':
                gradients = _x_adv.grad.sign()*step
            else:
                # .view() assumes batched image data as 4D tensor
                gradients = _x_adv.grad * step / _x_adv.grad.view(_x_adv.shape[0], -1).norm(step_norm, dim=-1)\
                    .view(-1, 1, 1, 1)

            if targeted:
                # Targeted: Gradient descent with on the loss of the (incorrect) target label
                # w.r.t. the model parameters
                x_adv -= gradients
            else:
                # Untargeted: Gradient ascent on the loss of the correct label w.r.t.
                # the model parameters
                x_adv += gradients


        # Project back into l_norm ball and correct range
        x_adv = project(x, x_adv, norm, eps)

    return x_adv.detach()

def pgd(model: Module,
        x: torch.Tensor,
        y: torch.Tensor,
        loss_fn: Callable,
        k: int,
        step: float,
        eps: float,
        norm: Union[str, float],
        y_target: torch.Tensor = None,
        random: bool = False) -> torch.Tensor:
   
    return _iterative_gradient(model=model, x=x, y=y, loss_fn=loss_fn, k=k, eps=eps, norm=norm, step=step, step_norm=2,
                               y_target=y_target, random=random)


def project(x: torch.Tensor, x_adv: torch.Tensor, norm: Union[str, int], eps: float) -> torch.Tensor:
    
    if x.shape != x_adv.shape:
        raise ValueError('Input Tensors must have the same shape')

    if norm == 'inf':
        # Workaround as PyTorch doesn't have elementwise clip
        x_adv = torch.max(torch.min(x_adv, x + eps), x - eps)
    else:
        delta = x_adv - x

        # Assume x and x_adv are batched tensors where the first dimension is
        # a batch dimension
        mask = delta.view(delta.shape[0], -1).norm(norm, dim=1) <= eps

        scaling_factor = delta.view(delta.shape[0], -1).norm(norm, dim=1)
        scaling_factor[mask] = eps

        # .view() assumes batched images as a 4D Tensor
        delta *= eps / scaling_factor.view(-1, 1, 1, 1)

        x_adv = x + delta

    return x_adv

In [ ]:
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'cifar'
image_datasets = {x: datasets.CIFAR10(data_dir, train = (x == 'train'), download=False,
                                          transform=data_transforms[x])
                  for x in ['test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['test']}
class_names = image_datasets['test'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model_nonrobust = models.resnet18()
num_ftrs = model_nonrobust.fc.in_features
model_nonrobust.fc = nn.Linear(num_ftrs, len(class_names))
model_nonrobust.load_state_dict(torch.load("models/pwny_cifar_eps_0.pth"))
model_ft = model_nonrobust.to(device)
model_nonrobust.eval()

model_robust = models.resnet18()
num_ftrs = model_robust.fc.in_features
model_robust.fc = nn.Linear(num_ftrs, len(class_names))
model_robust.load_state_dict(torch.load("models/pwny_cifar_eps_0.5.pth"))
model_ft = model_robust.to(device)
model_robust.eval()

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
#actual solution lol, the rest of this serves images
running_corrects = 0
criterion = nn.CrossEntropyLoss()
for i, (inputs, labels) in enumerate(dataloaders['test']):
    inputs = inputs.to(device)
    labels = labels.to(device)
    inputs = pgd(model_nonrobust, inputs, labels, criterion, k=15, step=0.1, eps=0.4, norm=2)
    out = torchvision.utils.make_grid(inputs)
#     imshow(out.cpu())
    inputs_r = inputs.clone().detach()
    outputs = model_nonrobust(inputs)
    outputs_r = model_robust(inputs_r)
    preds = torch.argmax(outputs, 1)
    preds_r = torch.argmax(outputs_r, 1)
    running_corrects += torch.sum(preds != preds_r) 
epoch_acc = running_corrects.double() / dataset_sizes['test']
print(epoch_acc)